# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [49]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

## Question 1:
Question 1: What is the sum of the outputs of the generator for limit = 5?


In [3]:
# to do: homework :)
# Question 1
limit = 5
generator = square_root_generator(limit)
sum = 0 
for sqrt_value in generator:
    sum+=sqrt_value
    print(sum)

1.0
2.414213562373095
4.146264369941973
6.146264369941973
8.382332347441762


In [47]:
# Question 1
import numpy as np
limit = 5
generator = square_root_generator(limit)
np.array(list(generator)).sum()

8.382332347441762

## Question 2:
Question 2: What is the 13th number yielded by the generator?

In [48]:
# question 2:
limit = 13
generator = square_root_generator(limit)
list(generator)[-1]

3.605551275463989

## Question 3:
Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.

In [51]:
# running the pipeline using dlt to duckdb, dataset name called homework, and table name call people
import duckdb

# We now use duckdb,
pipeline = dlt.pipeline(destination='duckdb', dataset_name='homework')

# we can load any generator to a table at the pipeline destnation as follows:
info = pipeline.run(people_1(),
                    table_name="people",
					write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

# we can load the next generator to the same table.
info = pipeline.run(people_2(),
                    table_name="people",
                    write_disposition="append")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)


Pipeline dlt_ipykernel_launcher load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////Users/kang/Documents/DE_zoomcamp/workshop_1_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707382402.1649148 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////Users/kang/Documents/DE_zoomcamp/workshop_1_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707382402.764307 is LOADED and contains no failed jobs


In [52]:
# Establish connection to duckdb and set path, showing the tables and query

import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# The answer for question 3

print("\n\n\n Sum of all ages of people after appending data:")

query = conn.sql("SELECT SUM(age) FROM people AS SUM_AGE").df()
display(query)

print("\n\n\n Table look like this after appending:")

query = conn.sql("SELECT * FROM people").df()
display(query)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 Sum of all ages of people after appending data:


,sum(age)
0,353.0





 Table look like this after appending:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707382402.1649148,TNHOyXBRB3Mk3Q,None
1,2,Person_2,27,City_A,1707382402.1649148,x/VM4F7FXCql5A,None
2,3,Person_3,28,City_A,1707382402.1649148,UR6hr9OMEWQf6w,None
3,4,Person_4,29,City_A,1707382402.1649148,0BlGLxL8iaNGDA,None
4,5,Person_5,30,City_A,1707382402.1649148,Lzby0LClXwRWvA,None
5,3,Person_3,33,City_B,1707382402.764307,kgLOgbTcMJih3Q,Job_3
6,4,Person_4,34,City_B,1707382402.764307,5ELSoLiYquQrzA,Job_4
7,5,Person_5,35,City_B,1707382402.764307,YtDqvLfIdueedQ,Job_5
8,6,Person_6,36,City_B,1707382402.764307,tNv2n9N701OdUQ,Job_6
9,7,Person_7,37,City_B,1707382402.764307,JPHMWL3WtlPu0Q,Job_7


In [32]:
query = conn.sql("SELECT * FROM people").df()
display(query)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707379941.4286358,+vAWUeTJmt8EyQ,None
1,2,Person_2,27,City_A,1707379941.4286358,kUNg++8zrXN+mQ,None
2,3,Person_3,28,City_A,1707379941.4286358,wHSkaDR0NVp+zA,None
3,4,Person_4,29,City_A,1707379941.4286358,mxC6Jk/+qge/LQ,None
4,5,Person_5,30,City_A,1707379941.4286358,dXhrcKytofNevA,None
5,3,Person_3,33,City_B,1707379942.025635,zkkypDkgJTIVTA,Job_3
6,4,Person_4,34,City_B,1707379942.025635,zRBfjLEkMTq2dQ,Job_4
7,5,Person_5,35,City_B,1707379942.025635,YiJFwGGv6YCr8A,Job_5
8,6,Person_6,36,City_B,1707379942.025635,Dg+uV93aeVpu+w,Job_6
9,7,Person_7,37,City_B,1707379942.025635,mL0jCiJR49jvBA,Job_7


## Question 4
Question 4. Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

In [53]:
# running the pipeline using dlt to duckdb, dataset name called homework, and table name call people
import duckdb

# We now use duckdb,
pipeline = dlt.pipeline(destination='duckdb', dataset_name='homework1')

# we can load any generator to a table at the pipeline destnation as follows:
info = pipeline.run(people_1(),
                    table_name="people1",
					write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

# we can load the next generator to the same table.
info = pipeline.run(people_2(),
                    table_name="people1",
                    write_disposition="merge",
                   primary_key="id")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework1
The duckdb destination used duckdb:////Users/kang/Documents/DE_zoomcamp/workshop_1_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707382584.8232682 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.30 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework1
The duckdb destination used duckdb:////Users/kang/Documents/DE_zoomcamp/workshop_1_dlt/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707382585.423342 is LOADED and contains no failed jobs


In [54]:
# Establish connection to duckdb and set path, showing the tables and query

import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# The answer for question 3

print("\n\n\n Sum of all ages of people after appending data:")

query = conn.sql("SELECT SUM(age) FROM people1 AS SUM_AGE").df()
display(query)

query = conn.sql("SELECT * FROM people1").df()
display(query)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people1             │
└─────────────────────┘




 Sum of all ages of people after appending data:


,sum(age)
0,266.0


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707382584.8232682,JeHZDEXji3u8kg,None
1,2,Person_2,27,City_A,1707382584.8232682,8/MmKh9Rlsy3lg,None
2,5,Person_5,35,City_B,1707382585.423342,vAcQsjncbUO/oA,Job_5
3,3,Person_3,33,City_B,1707382585.423342,0Jo/7FWgCLiQTg,Job_3
4,6,Person_6,36,City_B,1707382585.423342,cADFf4Gq7mP9RA,Job_6
5,4,Person_4,34,City_B,1707382585.423342,3lEroyw6Pmpjgw,Job_4
6,7,Person_7,37,City_B,1707382585.423342,FcOB9qIKuEYUKg,Job_7
7,8,Person_8,38,City_B,1707382585.423342,P6G2tEXQeWb/GA,Job_8


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707380056.988818,Vayq5lz58iXAcg,None
1,2,Person_2,27,City_A,1707380056.988818,OQaOlTTdzXZIMg,None
2,3,Person_3,33,City_B,1707380057.549649,vQOjG7aLJH6gBA,Job_3
3,6,Person_6,36,City_B,1707380057.549649,UqxqlzW8UC7qfw,Job_6
4,8,Person_8,38,City_B,1707380057.549649,EI5gVl4kgmzi/g,Job_8
5,7,Person_7,37,City_B,1707380057.549649,YeuORaN2iXGNpw,Job_7
6,5,Person_5,35,City_B,1707380057.549649,Jtr1oA1KipR5OA,Job_5
7,4,Person_4,34,City_B,1707380057.549649,kUKVS4v/o+SZew,Job_4


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX